<a href="https://colab.research.google.com/github/ddelizia/covid-19/blob/master/covid_19_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid Evolution Comunidad Valenciana

Data from: https://github.com/datadista/datasets/tree/master/COVID%2019

In [0]:
%pip install plotly

In [0]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [0]:
def normalize_data(dataset_url):
    df = pd.read_csv(dataset_url)
    df_ca = df.set_index('CCAA').drop(['cod_ine'], axis='columns').transpose()
    df_ca.index = pd.to_datetime(df_ca.index, format='%d/%m/%Y')
    return df_ca

df_cases = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv')
df_uci = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_uci.csv')
df_deaths = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv')
df_recovered = normalize_data('https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_altas.csv')

In [0]:
ca = 'C. Valenciana'

In [76]:
fig = make_subplots(rows=3, cols=3, 
                    subplot_titles=(
    "Cases", "Hist cases", "",
    "Daily Delta", "Recovered Delta", "Deaths Delta",
    "Daily Delta %", "Recovered Delta %",  "Deaths Delta %"))

fig.add_trace(go.Scatter(y=df_cases[ca], x=df_cases.index, name="All cases"), row=1, col=1)
fig.add_trace(go.Scatter(y=df_uci[ca], x=df_uci.index, name="Ucis"), row=1, col=1)
fig.add_trace(go.Scatter(y=df_deaths[ca], x=df_deaths.index, name="Deaths"), row=1, col=1)
fig.add_trace(go.Scatter(y=df_recovered[ca], x=df_recovered.index, name="Recovered"), row=1, col=1)

fig.add_trace(go.Bar(y=df_cases[ca], x=df_cases.index, name="All cases"), row=1, col=2)
fig.add_trace(go.Bar(y=df_uci[ca], x=df_uci.index, name="Ucis"), row=1, col=2)
fig.add_trace(go.Bar(y=df_deaths[ca], x=df_deaths.index, name="Deaths"), row=1, col=2)
fig.add_trace(go.Bar(y=df_recovered[ca], x=df_recovered.index, name="Recovered"), row=1, col=2)

fig.add_trace(go.Bar(y=df_cases[ca].diff(), x=df_cases.index, name="All case delta"), row=2, col=1)

fig.add_trace(go.Bar(y=100 * df_cases[ca].diff().pct_change(), x=df_cases.index, name="All cases delta %"), row=3, col=1)

fig.add_trace(go.Bar(y=df_recovered[ca].diff(), x=df_recovered.index, name="Recovered delta"), row=2, col=3)

fig.add_trace(go.Bar(y=100 * df_recovered[ca].diff().pct_change(), x=df_recovered.index, name="Recovered delta %"), row=3, col=3)

fig.add_trace(go.Bar(y=df_deaths[ca].diff(), x=df_deaths.index, name="Deaths delta"), row=2, col=2)

fig.add_trace(go.Bar(y=100 * df_deaths[ca].diff().pct_change(), x=df_deaths.index, name="Deaths delta %"), row=3, col=2)


fig.update_layout(height=1000,  title_text=f"Data {ca}")
fig.show()
py.iplot(fig, filename='multiple-subplots')



In [0]:
############# PIE

labels = ['All cases','Ucis','Deaths', 'Recovered']
values = [df_cases[ca][-1], df_uci[ca][-1], df_deaths[ca][-1], df_recovered[ca][-1]]

fig_pie = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig_pie.show()

In [87]:
fig_log = go.Figure()

fig_log.add_trace(go.Scatter(y=df_cases[ca], x=df_cases.index, name="All cases on log scale"))

fig_log.update_layout(yaxis_type="log", title_text=f"Log scale cases {ca}")
fig_log.show()